In [1]:
!pip install --user -r requirements.txt

In [2]:
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os

In [3]:
# Returns (R, G, B) from name
def name_to_color(name):
    # Take 3 first letters, tolower()
    # lowercased character ord() value rage is 97 to 122, substract 97, multiply by 8
    color = [((ord(c.lower())-97)*8) +10 for c in name[:3]]
    return color

In [4]:
# Function to resize images while maintaining image quality
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [5]:
# Modify Code for Video Stream image capture

KNOWN_FACES_DIR = 'dataset/Known_faces'
UNKNOWN_FACES_DIR = 'dataset/Unknown_faces'
TOLERANCE = 0.55 # Lower == More Strict
FRAME_THICKNESS = 3
FONT_THICKNESS = 1
MODEL = 'cnn'  # 'hog' or 'cnn' - CUDA accelerated (if available) deep-learning pretrained model

known_faces = []
known_names = []

In [6]:
# Evaluate and Encode Known Faces as base for comparison
print('Loading known faces...')
for name in os.listdir(KNOWN_FACES_DIR):
    # Next we load every file of faces of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        # Load an image
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        # Get 128-dimension face encoding
        # face_recognition always returns a list of found faces, for this purpose we take first face only
        # (assuming one face per image in Known_faces as a single person can't appear twice in one image)
        encoding = face_recognition.face_encodings(image)[0]
        # Append encodings and name
        known_faces.append(encoding)
        known_names.append(name)

Loading known faces...


In [ ]:
# Evaluate Unknown Faces Library
print('Processing unknown faces...')
# Now let's loop over a folder of faces we want to label
for filename in os.listdir(UNKNOWN_FACES_DIR):
    
    # Load image
    print(f'Filename {filename}', end='')
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}') # Grab image from directory
    locations = face_recognition.face_locations(image, model=MODEL) # grab face locations - we'll need them to draw boxes
    encodings = face_recognition.face_encodings(image, locations) # Pass face locations to be encoded so that model does not need to search again
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert it from RGB to BGR as we are going to work with cv2
    # We assume that there might be more faces in an image - let's find the faces of different people
    print(f', found {len(encodings)} face(s)')
    for face_encoding, face_location in zip(encodings, locations): 

        # We use compare_faces (but can use face_distance for distance-score measurement as well)
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE) # Returns array of True/False values in order of passed known_faces
        # Order is preserved, so we check if any face was found then grab its index, then label (name) of first matching known face withing a tolerance
        match = None
        if True in results:  # If at least one is true, get a name of first of found labels
            match = known_names[results.index(True)]
            print(f' - {match} from {results}')
            # Each location contains positions in order: top, right, bottom, left
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])
            # Get color by name using our fancy function
            color = name_to_color(match)
            # Paint frame
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)
            # Create a smaller, filled frame below for a name
            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2] + 22) # This time we use bottom in both corners - to start from bottom and move 50 pixels down
            # Paint frame
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)
            # Wite a name
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), FONT_THICKNESS)

    # Show image
    resize = ResizeWithAspectRatio(image, width=850) # Resize by width OR
#     resize = ResizeWithAspectRatio(image, height=1980) # Resize by height
    cv2.imshow(filename, resize)
    cv2.waitKey(0) 
    cv2.destroyWindow()